# 3rd Level Model Structure: Everything (Linking Stages)

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))

In [2]:
from pyomo import environ as pe
from modules.global_set import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule

from utility.display_utility import trans_product_mole, trans_product_mass, beautify2
from utility.model_utility import add_dual, update_dual, check_DOF

model = pe.ConcreteModel()

# Global Set

In [3]:
model.TRAY = pe.RangeSet(1,1)

# Construct Reactive Stages

In [4]:
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[1].T_F
| reactive[1].P
| reactive[1].cat
| reactive[1].Q_main
| reactive[1].x_
| reactive[1].y_
| reactive[1].x
| reactive[1].y
| reactive[1].z
| reactive[1].L
| reactive[1].V
| reactive[1].F
| reactive[1].H_L_
| reactive[1].H_V_
| reactive[1].H_L
| reactive[1].H_V
| reactive[1].T
| reactive[1].H_F
| reactive[1].f_V
| reactive[1].f_L
| reactive[1].r_total_comp
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[1].kinetics_block.k_FT
| reactive[1].kinetics_block.r_FT_total
| reactive[1].kinetics_block.g0_FT
| reactive[1].kinetics_block.alpha
| reactive[1].kinetics_block.r_FT_cnum
| reactive[1].kinetics_block.r_FT_comp
| reactive[1].kinetics_block.k_WGS
| reactive[1].kinetics_block.Ke_WGS
| reactive[1].kinetics_block.r_WGS
| reactive[1].kinetics_bloc

# Construct a single condenser

In [5]:
model.condenser = pe.Block(rule=condenser_stage_rule)

| Importing Condenser Stage......
| Adding the following local variable:
------------------------------------
| condenser.T
| condenser.T_F
| condenser.P
| condenser.Q_main
| condenser.x_
| condenser.y_
| condenser.x
| condenser.y
| condenser.z
| condenser.L
| condenser.W
| condenser.V
| condenser.F
| condenser.H_L_
| condenser.H_V_
| condenser.H_L
| condenser.H_V
| condenser.H_F
| condenser.f_V
| condenser.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.energy_block.dH_F
| condenser.energy_block.dH_V
| condenser.energy_block.dH_L
| condenser.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.VLE_block.n_ave
| condenser.VLE_block.n_ave_cal
| condenser.VLE_block.Hen
| condenser.VLE_block.Hen0
| condenser.VLE_blo

# Linking Stage Variables

### Vapor Between Reactive Stages

In [6]:
def V_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].V['in'] == model.reactive[j+1].V['out']
model.V_between_con = pe.Constraint(model.TRAY,rule=V_between_rule)

def Vy_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].y_['in',i] == model.reactive[j+1].y[i]
model.Vy_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Vy_between_rule)

def Vh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].H_V_['in'] == model.reactive[j+1].H_V
model.Vh_between_con = pe.Constraint(model.TRAY,rule=Vh_between_rule)

### Liquid Between Reactive Stages

In [7]:
def L_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].L['in'] == model.reactive[j].L['out']
model.L_between_con = pe.Constraint(model.TRAY,rule=L_between_rule)

def Lx_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].x_['in',i] == model.reactive[j].x[i]
model.Ly_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Lx_between_rule)

def Lh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].H_L_['in'] == model.reactive[j].H_L
model.Lh_between_con = pe.Constraint(model.TRAY,rule=Lh_between_rule)

### Condenser

In [8]:
def V_condenser_rule(model):
    return model.reactive[model.TRAY.first()].V['out'] == model.condenser.V['in']
model.V_condenser_con = pe.Constraint(rule=V_condenser_rule)

def Vy_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].y[i] == model.condenser.y_['in',i]
model.Vy_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_condenser_rule)

def Vh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_V == model.condenser.H_V_['in']
model.Vh_condenser_con = pe.Constraint(rule=Vh_condenser_rule)

In [9]:
def L_condenser_rule(model):
    return model.reactive[model.TRAY.first()].L['in'] == model.condenser.L['out']
model.L_condenser_con = pe.Constraint(rule=L_condenser_rule)

def Lx_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].x_['in',i] == model.condenser.x[i]
model.Lx_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_condenser_rule)

def Lh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_L_['in'] == model.condenser.H_L
model.Lh_condenser_con = pe.Constraint(rule=Lh_condenser_rule)

# Fixing Redundent Stream Variables

In [10]:
# condenser
model.condenser.VLE_block.n_ave.fix(58)

model.condenser.F.fix(0)
model.condenser.T_F.fix(300+273.15)
model.condenser.z.fix(0)

model.condenser.V['P'].fix(0)
model.condenser.L['in'].fix(0)
for i in m.COMP_TOTAL: model.condenser.x_['in',i].fix(0)
model.condenser.H_L_['in'].fix(0)

In [11]:
# 'reboiler' fixing last stage V_in

model.reactive[model.TRAY.last()].V['in'].fix(0)
for i in m.COMP_TOTAL: model.reactive[model.TRAY.last()].y_['in',i].fix(0)
model.reactive[model.TRAY.last()].H_V_['in'].fix(0)

In [12]:
check_DOF(pe,model)

Active Equality Constraints:	 1980
Active Inequality Constraints:	 0
Active Variables:		 2163
Fixed Variables:		 169
DOF:				 14


# Load Operating Parameters

In [13]:
# condenser
model.condenser.P.fix(19)
#model.condenser.T.fix(30+273.15)
model.condenser.T.setub(30+273.15)
model.condenser.L['out'].fix(7)

# reactive stage
for j in model.reactive:
    model.reactive[j].cat.fix(2000)
    model.reactive[j].P.fix(20)
    model.reactive[j].VLE_block.n_ave.fix(58)
    
    model.reactive[j].F.fix(2)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].z['CO'].fix(0.4)
    model.reactive[j].z['H2'].fix(0.6)
    model.reactive[j].z['C30H62'].fix(0)
    model.reactive[j].V['P'].fix(0)
    model.reactive[j].L['P'].fix(0)
    model.reactive[j].Q_main.fix(0)
    # model.reactive[j].T.setlb(270+273.15)

In [14]:
check_DOF(pe,model)

Active Equality Constraints:	 1980
Active Inequality Constraints:	 0
Active Variables:		 2163
Fixed Variables:		 182
DOF:				 1


In [15]:
model.obj = pe.Objective(expr = 1 ,sense=pe.maximize)

In [16]:
opt = pe.SolverFactory('ipopt')
opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'
# opt.options['nlp_scaling_method'] = None
# opt.options['constr_viol_tol'] = 1e-7
# opt.options['acceptable_constr_viol_tol'] = 1e-7
opt.options['max_iter'] = 7000
# opt.options['dual_inf_tol'] = '+inf'
# opt.options['acceptable_dual_inf_tol'] = '+inf'

In [17]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


In [18]:
# with open('../saved_solutions/1_stage_condenser.pickle', 'rb') as f:
#         results = pickle.load(f)

In [19]:
# opt.options['warm_start_init_point'] = 'yes'
# opt.options['warm_start_bound_push'] = 1e-20
# opt.options['warm_start_mult_bound_push'] = 1e-20
# opt.options['mu_init'] = 1e-6

In [20]:
# model.solutions.load_from(results)

In [21]:
# model.ipopt_zL_in.update(model.ipopt_zL_out)
# model.ipopt_zU_in.update(model.ipopt_zU_out)

In [22]:
results = opt.solve(model,tee=True)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
max_iter=7000


List of user-set options:

                                    Name   Value                used
                           linear_solver = ma86                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver ma86.

Number of nonzeros in equality constraint Jacobian...:     6544
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.....

  74r-1.0000000e+00 1.10e+02 9.99e+02   1.1 0.00e+00    -  0.00e+00 4.79e-08R  2
  75r-1.0000000e+00 1.10e+02 9.99e+02   1.1 4.36e+03    -  9.30e-05 6.46e-08f  2
  76r-1.0000000e+00 1.07e+02 9.98e+02   0.4 4.84e+02    -  2.69e-03 1.01e-03f  1
  77r-1.0000000e+00 9.77e+01 1.61e+03  -0.3 9.79e+02    -  8.66e-03 2.24e-03f  1
  78r-1.0000000e+00 9.77e+01 9.99e+02   1.0 0.00e+00    -  0.00e+00 3.13e-07R  2
  79r-1.0000000e+00 9.77e+01 1.14e+03   1.0 2.57e+03    -  2.00e-04 1.45e-09f  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80r-1.0000000e+00 9.06e+01 1.56e+03   0.3 5.51e+02    -  5.96e-03 2.08e-03f  1
  81r-1.0000000e+00 8.03e+01 2.50e+03  -0.4 1.12e+03    -  1.20e-02 2.80e-03f  1
  82r-1.0000000e+00 8.03e+01 9.99e+02   0.9 0.00e+00    -  0.00e+00 4.08e-07R  3
  83r-1.0000000e+00 8.03e+01 3.31e+03   0.9 1.69e+03    -  4.79e-04 1.50e-09f  3
  84r-1.0000000e+00 7.48e+01 4.38e+03   0.9 7.22e+02    -  2.02e-02 2.17e-03f  1
  85r-1.0000000e+00 5.09e+01

 189 -1.0000000e+00 1.56e-01 4.77e+04  -1.0 9.19e+02    -  3.93e-04 1.06e-04f  9
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190 -1.0000000e+00 1.56e-01 4.78e+04  -1.0 9.19e+02    -  5.41e-04 1.07e-04f  9
 191 -1.0000000e+00 1.56e-01 1.78e+05  -1.0 9.19e+02    -  1.34e-02 1.08e-04f  9
 192 -1.0000000e+00 1.56e-01 1.85e+05  -1.0 9.19e+02    -  7.46e-04 1.08e-04f  9
 193 -1.0000000e+00 1.56e-01 3.52e+05  -1.0 9.18e+02    -  1.57e-02 5.44e-05h 10
 194 -1.0000000e+00 1.56e-01 3.58e+05  -1.0 9.18e+02    -  9.27e-04 4.37e-04f  7
 195 -1.0000000e+00 1.56e-01 7.00e+05  -1.0 9.18e+02    -  2.98e-02 5.58e-05h 10
 196 -1.0000000e+00 1.56e-01 7.18e+05  -1.0 9.18e+02    -  1.42e-03 5.60e-05h 10
 197 -1.0000000e+00 1.56e-01 1.20e+06  -1.0 9.18e+02    -  3.59e-02 5.61e-05h 10
 198 -1.0000000e+00 1.56e-01 1.22e+06  -1.0 9.18e+02    -  1.53e-03 5.63e-05h 10
 199 -1.0000000e+00 1.50e+00 1.69e+06  -1.0 9.18e+02    -  6.06e-02 2.89e-02w  1
iter    objective    inf_pr 

 282 -1.0000000e+00 8.53e-02 2.64e+13  -1.0 9.19e+02    -  5.83e-03 3.06e-05h 11
 283 -1.0000000e+00 8.53e-02 3.18e+13  -1.0 9.19e+02    -  2.10e-02 3.07e-05h 11
 284 -1.0000000e+00 1.31e+00 2.56e+13  -1.0 9.19e+02    -  4.19e-03 3.14e-02w  1
 285 -1.0000000e+00 1.31e+00 5.44e+13  -1.0 8.41e+02    -  7.81e-02 6.25e-04w  1
 286 -1.0000000e+00 1.31e+00 2.60e+16  -1.0 8.38e+02    -  3.91e-02 6.36e-06w  1
 287 -1.0000000e+00 8.53e-02 3.34e+13  -1.0 2.54e-04  16.0 4.19e-03 3.07e-05h 10
 288 -1.0000000e+00 8.53e-02 6.29e+13  -1.0 9.19e+02    -  7.33e-02 3.07e-05h 11
 289 -1.0000000e+00 8.53e-02 6.57e+13  -1.0 9.19e+02    -  3.73e-03 3.07e-05h 11
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 290 -1.0000000e+00 8.53e-02 9.37e+13  -1.0 9.19e+02    -  3.54e-02 3.08e-05h 11
 291 -1.0000000e+00 8.53e-02 9.83e+13  -1.0 9.18e+02    -  4.10e-03 3.08e-05h 11
 292 -1.0000000e+00 8.53e-02 1.28e+15  -1.0 9.18e+02    -  1.00e+00 3.08e-05h 11
 293 -1.0000000e+00 8.53e-02

 377 -1.0000000e+00 1.14e+00 2.76e+16  -1.0 2.54e-03  15.3 4.22e-01 1.12e-01w  1
 378 -1.0000000e+00 8.51e-02 4.57e+16  -1.0 8.30e+02    -  7.08e-03 3.24e-05h 10
 379 -1.0000000e+00 8.51e-02 4.57e+16  -1.0 9.15e+02    -  1.12e-01 3.24e-05h 11
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 380 -1.0000000e+00 8.51e-02 4.57e+16  -1.0 9.14e+02    -  5.92e-03 3.24e-05h 11
 381 -1.0000000e+00 8.51e-02 4.57e+16  -1.0 9.14e+02    -  3.87e-02 3.24e-05h 11
 382 -1.0000000e+00 8.51e-02 4.57e+16  -1.0 9.14e+02    -  6.88e-03 3.25e-05h 11
 383 -1.0000000e+00 8.51e-02 4.57e+16  -1.0 9.14e+02    -  8.43e-02 3.25e-05h 11
 384 -1.0000000e+00 8.51e-02 4.57e+16  -1.0 9.14e+02    -  6.18e-03 3.25e-05h 11
 385 -1.0000000e+00 8.51e-02 4.57e+16  -1.0 9.14e+02    -  4.57e-02 3.25e-05h 11
 386 -1.0000000e+00 8.51e-02 4.57e+16  -1.0 9.14e+02    -  6.77e-03 3.25e-05h 11
 387 -1.0000000e+00 8.51e-02 4.57e+16  -1.0 9.14e+02    -  6.65e-02 3.26e-05h 11
 388 -1.0000000e+00 1.12e+00

...

1406 -1.0000000e+00 8.28e-02 6.88e+16  -1.0 8.33e+02    -  2.40e-01 2.48e-05h  9
1407 -1.0000000e+00 8.28e-02 6.88e+16  -1.0 8.33e+02    -  3.28e-02 2.47e-05h  9
1408 -1.0000000e+00 8.28e-02 6.88e+16  -1.0 8.33e+02    -  1.77e-01 2.46e-05h  9
1409 -1.0000000e+00 8.28e-02 6.88e+16  -1.0 8.33e+02    -  3.48e-02 2.45e-05h  9
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1410 -1.0000000e+00 8.27e-02 6.88e+16  -1.0 8.32e+02    -  2.22e-01 2.44e-05h  9
1411 -1.0000000e+00 8.27e-02 6.88e+16  -1.0 8.32e+02    -  3.33e-02 2.43e-05h  9
1412 -1.0000000e+00 8.27e-02 6.88e+16  -1.0 8.32e+02    -  1.90e-01 2.42e-05h  9
1413 -1.0000000e+00 8.27e-02 6.88e+16  -1.0 8.32e+02    -  3.44e-02 2.41e-05h  9
1414 -1.0000000e+00 8.27e-02 6.88e+16  -1.0 8.32e+02    -  2.06e-01 2.40e-05h  9
1415 -1.0000000e+00 8.55e-02 6.88e+16  -1.0 8.32e+02    -  3.38e-02 6.12e-03w  1
1416 -1.0000000e+00 8.55e-02 6.88e+16  -1.0 8.11e+02    -  1.77e-01 5.73e-05w  1
1417 -1.0000000e+00 8.55e-02

1499 -1.0000000e+00 8.26e-02 6.88e+16  -1.0 8.25e+02    -  1.07e-01 3.05e-05h  8
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1500 -1.0000000e+00 8.26e-02 6.88e+16  -1.0 8.25e+02    -  4.77e-02 3.03e-05h  8
1501 -1.0000000e+00 8.26e-02 6.88e+16  -1.0 8.25e+02    -  1.00e+00 3.00e-05h  8
1502 -1.0000000e+00 8.26e-02 6.88e+16  -1.0 8.24e+02    -  3.11e-02 2.98e-05h  8
1503 -1.0000000e+00 8.26e-02 6.88e+16  -1.0 8.24e+02    -  9.81e-02 2.95e-05h  8
1504 -1.0000000e+00 8.26e-02 6.88e+16  -1.0 8.24e+02    -  5.02e-02 2.93e-05h  8
1505 -1.0000000e+00 8.26e-02 6.88e+16  -1.0 8.24e+02    -  1.00e+00 2.91e-05h  8
1506 -1.0000000e+00 8.34e-02 6.88e+16  -1.0 8.24e+02    -  3.12e-02 3.69e-03w  1
1507 -1.0000000e+00 8.34e-02 6.88e+16  -1.0 8.11e+02    -  1.04e-01 3.55e-05w  1
1508 -1.0000000e+00 8.34e-02 6.87e+16  -1.0 1.65e-04  15.8 1.16e-03 1.16e-03w  1
1509 -1.0000000e+00 8.26e-02 6.88e+16  -1.0 9.31e-05  16.2 3.12e-02 2.88e-05h  7
iter    objective    inf_pr 

1591 -1.0000000e+00 8.24e-02 6.88e+16  -1.0 8.18e+02    -  3.24e-02 2.87e-05h  7
1592 -1.0000000e+00 8.24e-02 6.88e+16  -1.0 8.18e+02    -  1.11e-01 2.83e-05h  7
1593 -1.0000000e+00 8.24e-02 6.88e+16  -1.0 8.17e+02    -  4.93e-02 2.78e-05h  7
1594 -1.0000000e+00 8.24e-02 6.88e+16  -1.0 8.17e+02    -  1.00e+00 2.74e-05h  7
1595 -1.0000000e+00 8.24e-02 6.88e+16  -1.0 8.17e+02    -  3.24e-02 2.69e-05h  7
1596 -1.0000000e+00 8.24e-02 6.88e+16  -1.0 8.17e+02    -  1.05e-01 2.65e-05h  7
1597 -1.0000000e+00 8.25e-02 6.88e+16  -1.0 8.17e+02    -  5.11e-02 1.67e-03w  1
1598 -1.0000000e+00 8.25e-02 6.88e+16  -1.0 2.93e-04  15.4 6.88e-04 6.88e-04w  1
1599 -1.0000000e+00 8.25e-02 6.88e+16  -1.0 8.11e+02    -  4.20e-02 1.64e-05w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1600 -1.0000000e+00 8.24e-02 6.88e+16  -1.0 4.92e-05  16.7 5.11e-02 2.61e-05h  6
1601 -1.0000000e+00 8.24e-02 6.88e+16  -1.0 8.17e+02    -  1.00e+00 2.57e-05h  7
1602 -1.0000000e+00 8.24e-02

1688r-1.0000000e+00 2.17e-02 9.95e+02  -1.0 0.00e+00    -  0.00e+00 4.82e-07R 12
1689r-1.0000000e+00 2.21e-02 9.91e+02  -1.0 1.90e+02    -  4.63e-02 6.12e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1690r-1.0000000e+00 2.19e-02 1.46e+03  -1.0 8.40e+01    -  1.31e-01 5.60e-02f  1
1691r-1.0000000e+00 3.79e-01 2.24e+03  -1.0 4.75e+01    -  5.25e-01 2.22e-01f  1
1692r-1.0000000e+00 6.68e-01 8.37e+02  -1.0 3.63e+01    -  5.10e-01 6.95e-01f  1
1693r-1.0000000e+00 4.15e-01 7.97e+01  -1.0 8.41e+00    -  9.43e-01 1.00e+00f  1
1694r-1.0000000e+00 2.76e-01 3.31e+02  -1.7 6.72e+00    -  8.09e-01 5.92e-01f  1
1695r-1.0000000e+00 2.71e+00 1.76e+02  -1.7 1.53e+01    -  9.75e-01 9.37e-01f  1
1696r-1.0000000e+00 2.04e-01 2.25e+01  -1.7 4.06e+00    -  1.00e+00 1.00e+00f  1
1697r-1.0000000e+00 2.57e+00 2.65e+00  -1.7 3.27e+00    -  1.00e+00 1.00e+00h  1
1698r-1.0000000e+00 8.34e-02 1.10e+00  -1.7 2.92e+00    -  1.00e+00 1.00e+00h  1
1699r-1.0000000e+00 2.52e+00

	message from solver=Ipopt 3.12.8\x3a Converged to a locally infeasible point. Problem may be infeasible.


In [ ]:
model.ipopt_zL_in.update(model.ipopt_zL_out)
model.ipopt_zU_in.update(model.ipopt_zU_out)

In [ ]:
beautify2(pe,model)

In [ ]:
# model.solutions.store_to(results)
# with open('../saved_solutions/1_stage_condenser_n58_tmp.pickle','wb') as f:
#     pickle.dump(results,f)

### Check non-active constraints (dual = 0)

In [ ]:
for i in model.dual:
    if abs(model.dual[i]) == 0:
        print('{:40s}'.format(str(i)),'\t\t',model.dual[i])

# Iterative Data Analysis

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
opt.options['linear_solver'] = 'ma86'

In [ ]:
model.del_component(model.obj)
model.obj = pe.Objective(expr = model.condenser.L['P'] ,sense=pe.maximize)
model.condenser.T.fixed = True
model.condenser.L['out'].fixed=True
for j in model.reactive:
    model.reactive[j].Q_main.fixed=False
    model.reactive[j].T.fixed=True

In [ ]:
results = opt.solve(model,tee=True)

In [ ]:
update_dual(pe,model)

In [ ]:
Trange = np.linspace(model.reactive[1].T.value,200+273.15,50)

In [ ]:
rf_data = {}
rf_data['r'] = {}; rf_data['T'] = []; rf_data['Q'] = []; rf_data['V'] = []; rf_data['L'] = []; rf_data['Re'] = []; 
rf_data['y_CO'] = []; rf_data['y_H2'] = []; rf_data['y_CO2'] = []; rf_data['y_H2O'] = []; rf_data['r_WGS'] = []; rf_data['r_FT'] = []

for i in model.reactive[1].r_total_comp:
    rf_data['r'][i] = []

for t in Trange:
    model.reactive[1].T.fix(t)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    print('Solved, Solution T = {} K'.format(model.reactive[1].T.value))
    rf_data['T'].append(model.reactive[1].T.value)
    rf_data['Q'].append(model.reactive[1].Q_main.value)
    rf_data['V'].append(model.condenser.V['out'].value)
    rf_data['L'].append(model.condenser.L['P'].value)
    rf_data['Re'].append(model.condenser.L['out'].value)

    for i in model.reactive[1].r_total_comp:
        rf_data['r'][i].append(model.reactive[1].r_total_comp[i].value)
    
    
    rf_data['y_H2O'].append(model.reactive[1].y['H2O'].value)
    rf_data['y_CO'].append(model.reactive[1].y['CO'].value)
    rf_data['y_H2'].append(model.reactive[1].y['H2'].value)
    rf_data['y_CO2'].append(model.reactive[1].y['CO2'].value)
    rf_data['r_WGS'].append(model.reactive[1].kinetics_block.r_WGS.value)
    rf_data['r_FT'].append(model.reactive[1].kinetics_block.r_FT_total.value)

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,9))
line1 = ax1.plot(rf_data['T'],rf_data['Q'], 'b-')
ax1.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax1.set_ylabel('Q (MW)', color='b',fontsize=18)
ax1.tick_params('y', colors='b',labelsize=18)
ax1.tick_params('x', colors='k',labelsize=18)
ax1.grid()

ax2 = ax1.twinx()
line2 = ax2.plot(rf_data['T'],rf_data['r_FT'], 'r')
line3 = ax2.plot(rf_data['T'],rf_data['r_WGS'], 'r--')
ax2.set_ylabel('$r_{FT}$: (kmol/s Carbon Number)', color='r',fontsize=18)
ax2.tick_params('y', colors='r',labelsize=18)

ax1.legend(line1+line2+line3,['Q','$r_{FT}$','$r_{WGS}$'],fontsize=18,loc=5)
ax1.set_title('Heat Removal Rate and Reaction Rate vs T',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(rf_data['T'],rf_data['y_CO'])
ax.plot(rf_data['T'],rf_data['y_H2'])
ax.plot(rf_data['T'],rf_data['y_CO2'])
ax.plot(rf_data['T'],rf_data['y_H2O'])

ax.legend(['$y_{CO}$','$y_{H2}$','$y_{CO2}$','$y_{H2O}$'],fontsize=18)
ax.set_title('Gas Phase Inorganic Composition vs T',fontsize=18)

ax.set_ylabel('Mole Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax.tick_params('y', colors='k',labelsize=18)
ax.tick_params('x', colors='k',labelsize=18)
ax.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(rf_data['T'],rf_data['r']['CO'])
ax.plot(rf_data['T'],rf_data['r']['H2'])
ax.plot(rf_data['T'],rf_data['r']['CO2'])
ax.plot(rf_data['T'],rf_data['r']['H2O'])

ax.legend(['$r_{CO}$','$r_{H2}$','$r_{CO2}$','$r_{H2O}$'],fontsize=18)
ax.set_title('Inorganic Reaction Rate vs T',fontsize=18)

ax.set_ylabel('Reaction Rate (kmol/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax.tick_params('y', colors='k',labelsize=18)
ax.tick_params('x', colors='k',labelsize=18)
ax.grid()
plt.show()

In [ ]:
reaction_data = trans_product_mole(rf_data['r'])['unscaled']
reaction_data_scaled = trans_product_mole(rf_data['r'])['scaled']

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], reaction_data_scaled['c1'], reaction_data_scaled['c2'],reaction_data_scaled['c3'],reaction_data_scaled['c4'],\
            reaction_data_scaled['napha'],reaction_data_scaled['gasoline'],reaction_data_scaled['diesel'],reaction_data_scaled['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Product Composition (Mole) vs T',fontsize=18)
ax.set_ylabel('Mole Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], reaction_data['c1'], reaction_data['c2'],reaction_data['c3'],reaction_data['c4'],\
            reaction_data['napha'],reaction_data['gasoline'],reaction_data['diesel'],reaction_data['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Total Production Rate (Mole) vs T',fontsize=18)
ax.set_ylabel('Reaction Rate (kmol/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
mass_data = {}
for i in m.COMP_ORG:
    mass_data[i] = np.array(rf_data['r'][i])*u.cal_MW(i)

In [ ]:
mass_reaction_data = u.trans_product(mass_data)
mass_reaction_scaled = u.trans_product_scaled(mass_data)

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], mass_reaction_scaled['c1'], mass_reaction_scaled['c2'],mass_reaction_scaled['c3'],mass_reaction_scaled['c4'],\
            mass_reaction_scaled['napha'],mass_reaction_scaled['gasoline'],mass_reaction_scaled['diesel'],mass_reaction_scaled['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Product Composition (Mass) vs T',fontsize=18)
ax.set_ylabel('Mass Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], mass_reaction_data['c1'], mass_reaction_data['c2'],mass_reaction_data['c3'],mass_reaction_data['c4'],\
            mass_reaction_data['napha'],mass_reaction_data['gasoline'],mass_reaction_data['diesel'],mass_reaction_data['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Total Production Rate (mass) vs T',fontsize=18)
ax.set_ylabel('Reaction Rate (kg/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()